In [1]:
!pip install typing spacy matplotlib textdescriptives ripser transformers accelerate torch numpy pandas tqdm networkx hf_transfer datasets seaborn pyarrow fastparquet datasets
!python -m spacy download en_core_web_sm
!pip install dcor

  Using cached spacy-3.8.11-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (27 kB)
  Using cached matplotlib-3.10.7-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (11 kB)
  Using cached textdescriptives-2.8.4-py3-none-any.whl.metadata (25 kB)
  Using cached ripser-0.6.12-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.7 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.11.0-py3-none-any.whl.metadata (19 kB)
  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached hf_transfer-0.1.9-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.7 kB)
  Using cached datasets-4.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached fastparquet-2024.11.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached mu

In [2]:
from huggingface_hub import login
login()

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import pandas as pd
import ripser
from ripser import ripser
from tqdm import tqdm
import math
import numpy as geek
import networkx as nx
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Optional, List, Tuple
import spacy
import textdescriptives as td
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import dcor

In [4]:

# ============================================================================
# TDA COMPUTATION FUNCTIONS
# ============================================================================

def find_highest_finite_value(data: np.ndarray) -> float:
    """Find the maximum finite value in an array"""
    finite_values = data[np.isfinite(data)]
    return np.max(finite_values) if len(finite_values) > 0 else 0.0


def get_second_value_ignoring_inf(data: np.ndarray) -> float:
    """Get the second highest finite value in an array"""
    non_inf_values = data[np.isfinite(data)]
    if len(non_inf_values) < 2:
        return 0.0
    sorted_values = np.sort(non_inf_values)[::-1]
    return sorted_values[1]


def get_llama_attention(text: str, model, tokenizer, max_length: int = 512) -> np.ndarray:
    """
    Extract attention matrix from a transformer model
    
    Args:
        text: Input text
        model: Transformer model
        tokenizer: Model tokenizer
        max_length: Maximum sequence length
        
    Returns:
        Symmetrized attention matrix (numpy array)
    """
    inputs = tokenizer(text, return_tensors='pt', truncation=True, 
                      padding=True, max_length=max_length)
    
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)
    
    # Average across all layers and heads
    attention_matrices = torch.stack(outputs.attentions).cpu()
    attention_matrix = attention_matrices.mean(dim=0).mean(dim=1).squeeze().numpy()
    
    # Remove self-attention and symmetrize
    np.fill_diagonal(attention_matrix, 0)
    attention_matrix = (attention_matrix + attention_matrix.T) / 2
    
    return attention_matrix


def build_graph(attention_matrix: np.ndarray, threshold: Optional[float] = None, 
                percentile: int = 50) -> nx.Graph:
    """
    Build a graph from an attention matrix using thresholding
    
    Args:
        attention_matrix: Square attention matrix
        threshold: Fixed threshold (if None, uses percentile)
        percentile: Percentile for automatic threshold
        
    Returns:
        NetworkX graph
    """
    graph = nx.Graph()
    num_nodes = attention_matrix.shape[0]
    graph.add_nodes_from(range(num_nodes))
    
    # Get upper triangle values
    upper_triangle_indices = np.triu_indices(num_nodes, k=1)
    upper_triangle_values = attention_matrix[upper_triangle_indices]
    
    # Filter non-zero values
    non_zero_values = upper_triangle_values[upper_triangle_values > 0]
    non_zero_values = non_zero_values[~np.isnan(non_zero_values)]
    
    if threshold is None:
        if len(non_zero_values) > 0:
            threshold = np.percentile(non_zero_values, percentile)
            if np.isnan(threshold) or threshold == 0:
                threshold = np.max(non_zero_values) * 0.1
        else:
            threshold = 0.0
    
    # Add edges above threshold
    edge_count = 0
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if attention_matrix[i, j] > threshold:
                graph.add_edge(i, j, weight=attention_matrix[i, j])
                edge_count += 1
    
    return graph


def compute_tda_features(graph: nx.Graph) -> List[float]:
    """
    Compute topological data analysis features from a graph
    
    Args:
        graph: NetworkX graph
        
    Returns:
        List of 12 TDA features
    """
    adjacency_matrix = nx.to_numpy_array(graph)
    
    # Check if graph is empty
    if adjacency_matrix.shape[0] == 0 or np.sum(adjacency_matrix) == 0:
        return [0.0] * 12
    
    # Convert similarity to distance
    distance_matrix = 1 - adjacency_matrix
    np.fill_diagonal(distance_matrix, 0)
    
    # Compute persistent homology
    try:
        diagrams = ripser(distance_matrix, maxdim=1, distance_matrix=True)['dgms']
    except Exception as e:
        print(f"Warning: Ripser failed - {e}")
        return [0.0] * 12
    
    h0 = diagrams[0]
    h1 = diagrams[1] if len(diagrams) > 1 else np.array([])
    
    # H0 features
    h0_lifetimes = h0[:, 1] - h0[:, 0]
    h0_finite = h0_lifetimes[np.isfinite(h0_lifetimes)]
    
    num_h0 = np.count_nonzero(h0_finite)
    highest_h0 = find_highest_finite_value(h0_lifetimes) if num_h0 > 0 else 0.0
    second_highest_h0 = get_second_value_ignoring_inf(h0_lifetimes) if num_h0 > 1 else 0.0
    highest_minus_second_h0 = highest_h0 - second_highest_h0 if num_h0 > 1 else 0.0
    mean_h0 = np.mean(h0_finite) if num_h0 > 0 else 0.0
    
    # H0 persistence statistics
    h0_persistences = np.sort(h0_finite) if num_h0 > 0 else np.array([0.0])
    betti_curve_0 = len(h0_persistences)
    
    # Calculate persistence entropy correctly
    sum_persistence_0 = np.sum(h0_persistences)
    if sum_persistence_0 > 0:
        h0_probs = h0_persistences / sum_persistence_0
        persistence_entropy_0 = -np.sum(h0_probs * np.log(h0_probs + 1e-10))
    else:
        persistence_entropy_0 = 0.0
    
    # H1 features
    h1_lifetimes = h1[:, 1] - h1[:, 0] if len(h1) > 0 else np.array([])
    h1_finite = h1_lifetimes[np.isfinite(h1_lifetimes)] if len(h1_lifetimes) > 0 else np.array([])
    
    num_h1 = np.count_nonzero(h1_finite)
    highest_h1 = find_highest_finite_value(h1_lifetimes) if num_h1 > 0 else 0.0
    second_highest_h1 = get_second_value_ignoring_inf(h1_lifetimes) if num_h1 > 1 else 0.0
    highest_minus_second_h1 = highest_h1 - second_highest_h1 if num_h1 > 1 else 0.0
    mean_h1 = np.mean(h1_finite) if num_h1 > 0 else 0.0
    
    # H1 persistence statistics
    h1_persistences = np.sort(h1_finite) if num_h1 > 0 else np.array([0.0])
    betti_curve_1 = len(h1_persistences)
    
    sum_persistence_1 = np.sum(h1_persistences)
    if sum_persistence_1 > 0:
        h1_probs = h1_persistences / sum_persistence_1
        persistence_entropy_1 = -np.sum(h1_probs * np.log(h1_probs + 1e-10))
    else:
        persistence_entropy_1 = 0.0
    
    return [
        num_h0, highest_h0, highest_minus_second_h0, mean_h0, 
        betti_curve_0, persistence_entropy_0,
        num_h1, highest_h1, highest_minus_second_h1, mean_h1, 
        betti_curve_1, persistence_entropy_1
    ]


def process_texts(texts: List[str], model_name: str = "meta-llama/Llama-3.1-8B",
                  max_length: int = 512) -> pd.DataFrame:
    """
    Process texts to extract TDA features from attention patterns
    
    Args:
        texts: List of text strings
        model_name: HuggingFace model identifier
        max_length: Maximum sequence length
        
    Returns:
        DataFrame with TDA features
    """
    print(f"Loading model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True,
        attn_implementation="eager"
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    data = []
    successful = 0
    failed = 0
    
    for i, text in enumerate(tqdm(texts, desc="Processing texts")):
        try:
            attention_matrix = get_llama_attention(text, model, tokenizer, max_length)
            print(f"\nText {i}: Attention matrix shape: {attention_matrix.shape}, non-zero: {np.count_nonzero(attention_matrix)}")
            
            graph = build_graph(attention_matrix, threshold=None, percentile=50)
            print(f"Text {i}: Graph has {graph.number_of_nodes()} nodes, {graph.number_of_edges()} edges")
            
            tda_features = compute_tda_features(graph)
            print(f"Text {i}: TDA features: {tda_features[:3]}...")
            
            data.append(tda_features)
            successful += 1
        except Exception as e:
            print(f"\n!!! ERROR processing text {i}: {type(e).__name__}: {e}")
            import traceback
            traceback.print_exc()
            data.append([0.0] * 12)
            failed += 1
    
    print(f"\n{'='*60}")
    print(f"Processing complete: {successful} successful, {failed} failed")
    print(f"{'='*60}")
    
    columns = [
        "Num_0dim", "Max_0dim", "Max_0dim_Minus_Second", "Mean_0dim", 
        "betti_curve_0", "persistence_entropy_0",
        "Num_1dim", "Max_1dim", "Max_1dim_Minus_Second", "Mean_1dim", 
        "betti_curve_1", "persistence_entropy_1"
    ]
    
    return pd.DataFrame(data, columns=columns)


In [5]:
# ============================================================================
# LINGUISTIC FEATURE EXTRACTION
# ============================================================================

def extract_linguistic_features(texts: pd.Series, nlp_model: str = "en_core_web_sm") -> pd.DataFrame:
    """
    Extract linguistic features using textdescriptives
    
    Args:
        texts: Series of text strings
        nlp_model: Spacy model name
        
    Returns:
        DataFrame with linguistic features
    """
    nlp = spacy.load(nlp_model)
    nlp.add_pipe("textdescriptives/all")
    
    results = []
    for text in tqdm(texts, desc="Extracting linguistic features"):
        try:
            doc = nlp(text)
            features = td.extract_df(doc)
            results.append(features.iloc[0])
        except Exception as e:
            print(f"Error processing text: {e}")
            # Add empty row with NaN values
            results.append(pd.Series())
    
    return pd.concat(results, axis=1).T.reset_index(drop=True)

In [6]:
# ============================================================================
# CORRELATION ANALYSIS
# ============================================================================

def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    """Clean dataframe by removing NaN/inf and converting to float"""
    df_clean = df.copy()
    df_clean = df_clean.replace([np.inf, -np.inf], np.nan)
    df_clean = df_clean.fillna(df_clean.mean())
    df_clean = df_clean.fillna(0)
    df_clean = df_clean.astype(float)
    return df_clean

def compute_dcor_matrix(tda_data, linguistic_data):
    tda_clean = clean_data(tda_data)
    ling_clean = clean_data(linguistic_data)
    
    tda_features = tda_clean.to_numpy()
    linguistic_features = ling_clean.to_numpy()
    
    corr_matrix = np.zeros((tda_features.shape[1], linguistic_features.shape[1]))
    for i in range(tda_features.shape[1]):
        for j in range(linguistic_features.shape[1]):
            try:
                corr_matrix[i, j] = dcor.distance_correlation(tda_features[:, i], linguistic_features[:, j])
            except Exception as e:
                print(f"Warning: Could not compute correlation for indices ({i}, {j}): {e}")
                corr_matrix[i, j] = 0.0
    
    return corr_matrix
    
def plot_category_heatmap(corr_matrix: np.ndarray, 
                          tda_feature_names: List[str],
                          linguistic_feature_names: List[str], 
                          title: str, 
                          figsize: Tuple[int, int] = (12, 6)) -> plt.Figure:
    """Plot heatmap for a specific category"""
    fig, ax = plt.subplots(figsize=figsize)
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt='.3f',
                xticklabels=linguistic_feature_names,
                yticklabels=tda_feature_names,
                vmin=0, vmax=1, ax=ax)
    ax.set_title(title, fontsize=14, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    return fig

In [7]:
# ============================================================================
# MAIN PIPELINE
# ============================================================================

# Feature categories for analysis
FEATURE_CATEGORIES = {
    'token_stats': [
        'token_length_mean', 'token_length_median', 'token_length_std',
        'n_tokens', 'n_unique_tokens', 'proportion_unique_tokens',
        'mean_word_length', 'alpha_ratio'
    ],
    'sentence_stats': [
        'sentence_length_mean', 'sentence_length_median', 'sentence_length_std',
        'n_sentences', 'doc_length'
    ],
    'syllable_stats': [
        'syllables_per_token_mean', 'syllables_per_token_median', 
        'syllables_per_token_std'
    ],
    'pos_proportions': [
        'pos_prop_ADJ', 'pos_prop_ADP', 'pos_prop_ADV', 'pos_prop_AUX',
        'pos_prop_CCONJ', 'pos_prop_DET', 'pos_prop_INTJ', 'pos_prop_NOUN',
        'pos_prop_NUM', 'pos_prop_PART', 'pos_prop_PRON', 'pos_prop_PROPN',
        'pos_prop_PUNCT', 'pos_prop_SCONJ', 'pos_prop_SYM', 'pos_prop_VERB',
        'pos_prop_X'
    ],
    'information_theory': [
        'entropy', 'perplexity', 'per_word_perplexity', 'oov_ratio'
    ],
    'coherence': [
        'first_order_coherence', 'second_order_coherence'
    ],
    'dependency': [
        'dependency_distance_mean', 'dependency_distance_std',
        'prop_adjacent_dependency_relation_mean', 
        'prop_adjacent_dependency_relation_std'
    ],
    'readability': [
        'flesch_reading_ease', 'flesch_kincaid_grade', 'smog',
        'gunning_fog', 'automated_readability_index', 
        'coleman_liau_index', 'lix', 'rix'
    ]
}


def analyze_dataset(dataset_name: str,
                   dataset_config: str,
                   text_column: str,
                   label_column: str,
                   model_name: str = "meta-llama/Llama-3.1-8B",
                   n_samples: int = 500,
                   output_pdf: str = "TDA_Linguistic_Heatmaps.pdf",
                   tda_features: Optional[List[str]] = None):
    """
    Complete pipeline to analyze TDA-linguistic correlations
    
    Args:
        dataset_name: HuggingFace dataset name (e.g., "nyu-mll/glue")
        dataset_config: Dataset configuration (e.g., "cola")
        text_column: Name of text column in dataset
        label_column: Name of label column in dataset
        model_name: HuggingFace model identifier
        n_samples: Number of samples per label
        output_pdf: Output PDF filename
        tda_features: List of TDA features to use (default: H0 features)
    """
    
    # Default TDA features (H0 only for faster computation)
    if tda_features is None:
        tda_features = ["Num_0dim", "Max_0dim", "Mean_0dim", 
                       "betti_curve_0", "persistence_entropy_0"]
    
    print("="*60)
    print(f"TDA-Linguistic Analysis Pipeline")
    print(f"Dataset: {dataset_name}/{dataset_config}")
    print(f"Model: {model_name}")
    print("="*60)
    
    # Load dataset
    print("\n1. Loading dataset...")
    ds = load_dataset(dataset_name, dataset_config, split="train")
    df = ds.to_pandas()
    
    # Get unique labels
    unique_labels = df[label_column].unique()
    print(f"Found {len(unique_labels)} unique labels: {unique_labels}")
    
    # Process each label
    results = {}
    for label in unique_labels:
        print(f"\n{'='*60}")
        print(f"Processing label: {label}")
        print(f"{'='*60}")
        
        # Sample texts
        texts = df[df[label_column] == label][text_column].head(n_samples)
        print(f"Number of texts: {len(texts)}")
        
        # Extract TDA features
        print("\n2. Extracting TDA features...")
        tda_df = process_texts(texts.tolist(), model_name=model_name)
        
        # Extract linguistic features
        print("\n3. Extracting linguistic features...")
        linguistic_df = extract_linguistic_features(texts)
        
        results[label] = {
            'tda': tda_df,
            'linguistic': linguistic_df
        }
    
    # Create correlation heatmaps
    print(f"\n{'='*60}")
    print("4. Computing correlations and generating heatmaps...")
    print(f"{'='*60}")
    
    with PdfPages(output_pdf) as pdf:
        for category, features in FEATURE_CATEGORIES.items():
            print(f"\nProcessing category: {category}")
            
            # Get available features
            sample_ling = list(results.values())[0]['linguistic']
            available_features = [f for f in features if f in sample_ling.columns]
            
            if len(available_features) == 0:
                print(f"No features available for {category}, skipping...")
                continue
            
            # Process each label
            for label in unique_labels:
                tda_data = results[label]['tda'][tda_features]
                ling_data = results[label]['linguistic'][available_features]
                
                # Compute correlation
                corr_matrix = compute_dcor_matrix(tda_data, ling_data)
                
                # Plot heatmap
                fig = plot_category_heatmap(
                    corr_matrix,
                    tda_features,
                    available_features,
                    f"Label {label} - {category.replace('_', ' ').title()}"
                )
                pdf.savefig(fig, bbox_inches='tight')
                plt.close(fig)
            
            # If two labels, plot difference
            if len(unique_labels) == 2:
                labels = list(unique_labels)
                corr_0 = compute_dcor_matrix(
                    results[labels[0]]['tda'][tda_features],
                    results[labels[0]]['linguistic'][available_features]
                )
                corr_1 = compute_dcor_matrix(
                    results[labels[1]]['tda'][tda_features],
                    results[labels[1]]['linguistic'][available_features]
                )
                
                corr_diff = corr_0 - corr_1
                
                fig, ax = plt.subplots(figsize=(12, 6))
                sns.heatmap(corr_diff, annot=True, cmap="RdBu_r", fmt='.3f', 
                           center=0, xticklabels=available_features,
                           yticklabels=tda_features, vmin=-1, vmax=1, ax=ax)
                ax.set_title(f"Difference: Label {labels[0]} - Label {labels[1]} - "
                           f"{category.replace('_', ' ').title()}", 
                           fontsize=14, fontweight='bold')
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                pdf.savefig(fig, bbox_inches='tight')
                plt.close(fig)
    
    print(f"\n{'='*60}")
    print("ANALYSIS COMPLETE!")
    print(f"Heatmaps saved to: {output_pdf}")
    print(f"{'='*60}")
    
    return results

In [10]:
# if __name__ == "__main__":
#     # Example: CoLA dataset
#     results = analyze_dataset(
#         dataset_name="nyu-mll/glue",
#         dataset_config="cola",
#         text_column="sentence",
#         label_column="label",
#         model_name="meta-llama/Llama-3.1-8B",
#         n_samples=20,
#         output_pdf="CoLA_TDA_Linguistic_Heatmaps.pdf"
#     )

In [ ]:
"""
Run TDA-Linguistic analysis across all dataset-model combinations
"""

if __name__ == "__main__":
    
    # Define all models
    models = {
        "llama-3.1-8b-instruct": "meta-llama/Llama-3.1-8B-Instruct"
    }
    
    # models = {
    #     "llama-3.1-8b-instruct": "meta-llama/Llama-3.1-8B-Instruct",
    #     "qwen-2.5-7b-instruct": "Qwen/Qwen2.5-7B-Instruct",
    #     "gpt2": "gpt2",
    #     "distilgpt2": "distilgpt2",
    #     "roberta-base": "roberta-base",
    #     "bert-base-uncased": "bert-base-uncased",
    #     "distilbert-base-uncased": "distilbert-base-uncased",
    #     "electra-small": "google/electra-small-discriminator"
    # }
    
    # Define all dataset configurations
    datasets = [
        # GLUE benchmark
        # {
        #     "name": "nyu-mll/glue",
        #     "config": "cola",
        #     "text_column": "sentence",
        #     "label_column": "label",
        #     "description": "GLUE-CoLA"
        # },
        # {
        #     "name": "nyu-mll/glue",
        #     "config": "sst2",
        #     "text_column": "sentence",
        #     "label_column": "label",
        #     "description": "GLUE-SST2"
        # },
        # {
        #     "name": "nyu-mll/glue",
        #     "config": "mrpc",
        #     "text_column": "sentence1",  # Note: MRPC has two sentences
        #     "label_column": "label",
        #     "description": "GLUE-MRPC"
        # },
        # {
        #     "name": "nyu-mll/glue",
        #     "config": "qqp",
        #     "text_column": "question1",  # Note: QQP has two questions
        #     "label_column": "label",
        #     "description": "GLUE-QQP"
        # },
        {
            "name": "nyu-mll/glue",
            "config": "mnli",
            "text_column": "premise",  # Note: MNLI has premise and hypothesis
            "label_column": "label",
            "description": "GLUE-MNLI"
        },
        {
            "name": "nyu-mll/glue",
            "config": "qnli",
            "text_column": "question",  # Note: QNLI has question and sentence
            "label_column": "label",
            "description": "GLUE-QNLI"
        },
        {
            "name": "nyu-mll/glue",
            "config": "rte",
            "text_column": "sentence1",  # Note: RTE has two sentences
            "label_column": "label",
            "description": "GLUE-RTE"
        },
        {
            "name": "nyu-mll/glue",
            "config": "wnli",
            "text_column": "sentence1",  # Note: WNLI has two sentences
            "label_column": "label",
            "description": "GLUE-WNLI"
        },
        
        # SuperGLUE benchmark
        {
            "name": "super_glue",
            "config": "boolq",
            "text_column": "question",  # Note: BoolQ has passage and question
            "label_column": "label",
            "description": "SuperGLUE-BoolQ"
        },
        {
            "name": "super_glue",
            "config": "cb",
            "text_column": "premise",  # Note: CB has premise and hypothesis
            "label_column": "label",
            "description": "SuperGLUE-CB"
        },
        {
            "name": "super_glue",
            "config": "copa",
            "text_column": "premise",  # Note: COPA has premise and choices
            "label_column": "label",
            "description": "SuperGLUE-COPA"
        },
        {
            "name": "super_glue",
            "config": "wic",
            "text_column": "sentence1",  # Note: WiC has two sentences
            "label_column": "label",
            "description": "SuperGLUE-WiC"
        },
        {
            "name": "super_glue",
            "config": "wsc",
            "text_column": "text",
            "label_column": "label",
            "description": "SuperGLUE-WSC"
        },
        
        # SNLI
        {
            "name": "snli",
            "config": None,
            "text_column": "premise",  # Note: SNLI has premise and hypothesis
            "label_column": "label",
            "description": "SNLI"
        },
        
        # Multi-NLI
        {
            "name": "multi_nli",
            "config": None,
            "text_column": "premise",  # Note: MultiNLI has premise and hypothesis
            "label_column": "label",
            "description": "MultiNLI"
        },
        
        # BLiMP (Alex Warstadt's benchmark)
        {
            "name": "blimp",
            "config": "anaphor_agreement",
            "text_column": "sentence_good",  # BLiMP has good/bad sentence pairs
            "label_column": None,  # BLiMP doesn't have traditional labels
            "description": "BLiMP-Anaphor"
        },
        {
            "name": "blimp",
            "config": "argument_structure",
            "text_column": "sentence_good",
            "label_column": None,
            "description": "BLiMP-Argument"
        },
        
        # Winogrande
        {
            "name": "winogrande",
            "config": "winogrande_xl",
            "text_column": "sentence",
            "label_column": "answer",
            "description": "Winogrande-XL"
        },
    ]
    
    # Run all combinations
    import os
    from datetime import datetime
    
    results_dir = "tda_linguistic_results"
    os.makedirs(results_dir, exist_ok=True)
    
    log_file = os.path.join(results_dir, f"run_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")
    
    total_combinations = len(datasets) * len(models)
    current = 0
    
    with open(log_file, 'w') as log:
        log.write(f"Starting TDA-Linguistic Analysis\n")
        log.write(f"Total combinations: {total_combinations}\n")
        log.write(f"Datasets: {len(datasets)}\n")
        log.write(f"Models: {len(models)}\n")
        log.write("="*80 + "\n\n")
        
        for dataset_config in datasets:
            for model_name, model_path in models.items():
                current += 1
                
                # Skip BLiMP datasets without labels for now (needs special handling)
                if dataset_config["label_column"] is None:
                    print(f"[{current}/{total_combinations}] Skipping {dataset_config['description']} - {model_name} (needs special handling)")
                    log.write(f"[{current}/{total_combinations}] SKIPPED: {dataset_config['description']} - {model_name} (no labels)\n")
                    continue
                
                output_filename = f"{dataset_config['description']}_{model_name}_TDA_Linguistic.pdf"
                output_path = os.path.join(results_dir, output_filename)
                
                print(f"\n{'='*80}")
                print(f"[{current}/{total_combinations}] Processing:")
                print(f"  Dataset: {dataset_config['description']}")
                print(f"  Model: {model_name}")
                print(f"  Output: {output_filename}")
                print(f"{'='*80}\n")
                
                log.write(f"[{current}/{total_combinations}] STARTING: {dataset_config['description']} - {model_name}\n")
                log.flush()
                
                try:
                    # Run analysis
                    if dataset_config["config"] is not None:
                        results = analyze_dataset(
                            dataset_name=dataset_config["name"],
                            dataset_config=dataset_config["config"],
                            text_column=dataset_config["text_column"],
                            label_column=dataset_config["label_column"],
                            model_name=model_path,
                            n_samples=50,
                            output_pdf=output_path
                        )
                    else:
                        results = analyze_dataset(
                            dataset_name=dataset_config["name"],
                            dataset_config="default",  # Some datasets don't have configs
                            text_column=dataset_config["text_column"],
                            label_column=dataset_config["label_column"],
                            model_name=model_path,
                            n_samples=50,
                            output_pdf=output_path
                        )
                    
                    log.write(f"[{current}/{total_combinations}] SUCCESS: {dataset_config['description']} - {model_name}\n")
                    print(f"✓ Completed successfully: {output_filename}\n")
                    
                except Exception as e:
                    error_msg = f"ERROR: {str(e)}"
                    log.write(f"[{current}/{total_combinations}] FAILED: {dataset_config['description']} - {model_name} - {error_msg}\n")
                    print(f"✗ Failed: {error_msg}\n")
                    
                    # Continue with next combination
                    continue
                
                log.flush()
    
    print(f"\n{'='*80}")
    print("ALL COMBINATIONS COMPLETE!")
    print(f"Results saved to: {results_dir}/")
    print(f"Log file: {log_file}")
    print(f"{'='*80}")


[1/14] Processing:
  Dataset: GLUE-MNLI
  Model: llama-3.1-8b-instruct
  Output: GLUE-MNLI_llama-3.1-8b-instruct_TDA_Linguistic.pdf

TDA-Linguistic Analysis Pipeline
Dataset: nyu-mll/glue/mnli
Model: meta-llama/Llama-3.1-8B-Instruct

1. Loading dataset...


README.md: 0.00B [00:00, ?B/s]

mnli/train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

mnli/validation_matched-00000-of-00001.p(…):   0%|          | 0.00/1.21M [00:00<?, ?B/s]

mnli/validation_mismatched-00000-of-0000(…):   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...


Text 48: Attention matrix shape: (31, 31), non-zero: 930
Text 48: Graph has 31 nodes, 231 edges
Text 48: TDA features: [30, 0.694580078125, 0.03564453125]...

Text 49: Attention matrix shape: (18, 18), non-zero: 306
Text 49: Graph has 18 nodes, 76 edges
Text 49: TDA features: [17, 0.66162109375, 0.001708984375]...

Processing complete: 50 successful, 0 failed



3. Extracting linguistic features...


Extracting linguistic features:   0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/textdescriptives/components/coherence.py:44: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Span.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarities.append(sent.similarity(sents[i + order]))
Extracting linguistic features: 100%|██████████| 50/50 [00:00<00:00, 88.75it/s]



Processing label: 1
Number of texts: 50

2. Extracting TDA features...
Loading model: meta-llama/Llama-3.1-8B-Instruct


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]